In [4]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"  # ✅ Desactivar Numba JIT antes de todo

import pandas as pd
import numpy as np
import joblib
import vectorbt as vbt

# === 1. Cargar df_test ===
df_test = pd.read_csv("df_test.csv").reset_index(drop=True)

# Agregar columnas temporales si existe 'time'
if 'time' in df_test.columns:
    df_test['time'] = pd.to_datetime(df_test['time'])
    df_test['hour'] = df_test['time'].dt.hour
    df_test['weekday'] = df_test['time'].dt.weekday

# === 2. Cargar modelo entrenado ===
modelo_path = "best_model_walkforward_weekly_dt_fullparams.joblib"
model = joblib.load(modelo_path)
print(f"✅ Modelo cargado: {modelo_path}")

# === 3. Features para predicción ===
features_cols = [
    'close', 
    'sma_200_vs_100', 'sma_50_vs_25', 'rsi_14', 'acceleration_12',
    'macd', 'stoch_k', 'momentum_12', 'bb_middle', 'bb_upper',
    'bb_lower', 'weighted_close', 'poc_12', 'high_channel_20',
    'low_channel_20', 'abs_diff_mrb_vs_close',
    'last_signal_price_1', 'last_signal_price_2',
    'last_signal_price_3', 'last_signal_price_4',
    'last_signal_price_5', 'hour', 'weekday'
]

# Verificar columnas disponibles
features_cols = [col for col in features_cols if col in df_test.columns]
X_test = df_test[features_cols].fillna(0)

# === 4. Predicción binaria y probabilidades ===
df_test['pred'] = model.predict(X_test)
df_test['proba_1'] = model.predict_proba(X_test)[:, 1]

# === 5. Generar señales de entrada ===
entries = ((df_test['signal'] == 1) & (df_test['pred'] == 1)).fillna(False).astype(bool)

# === 6. Generar señales de salida (TP o después de N velas)
tp_pips = 0.00050
horizon = 48
exits = pd.Series(False, index=df_test.index)

for i in entries[entries].index:
    entry_price = df_test.at[i, 'close']
    future = df_test.iloc[i+1:min(i+1+horizon, len(df_test))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        fallback_idx = future.index[-1]
        exits.at[fallback_idx] = True

# === 7. Preprocesamiento final ===
prices = df_test['close'].astype(float).ffill().bfill()
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

# === 8. Configuración de trading ===
init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = float(lot_size * leverage)

# === 9. Crear portafolio con vectorbt ===
portfolio = vbt.Portfolio.from_signals(
    close=prices,
    entries=entries,
    exits=exits,
    size=fixed_size,
    init_cash=init_cash,
    fees=0.001,
    slippage=0.0005,
    direction='longonly'
)

# === 10. Estadísticas del portafolio ===
print("\n📊 Estadísticas del portafolio (con modelo):")
stats = portfolio.stats()

# Mostrar estadísticas clave
print(f"Start Value: {stats['Start Value']}")
print(f"End Value: {stats['End Value']}")
print(f"Total Return [%]: {stats['Total Return [%]']}")
print(f"Benchmark Return [%]: {stats['Benchmark Return [%]']}")
print(f"Max Gross Exposure [%]: {stats['Max Gross Exposure [%]']}")
print(f"Total Fees Paid: {stats['Total Fees Paid']}")
print(f"Max Drawdown [%]: {stats['Max Drawdown [%]']}")
print(f"Max Drawdown Duration: {stats['Max Drawdown Duration']}")
print(f"Total Trades: {stats['Total Trades']}")
print(f"Total Closed Trades: {stats['Total Closed Trades']}")
print(f"Total Open Trades: {stats['Total Open Trades']}")
print(f"Open Trade PnL: {stats['Open Trade PnL']}")
print(f"Win Rate [%]: {stats['Win Rate [%]']}")
print(f"Best Trade [%]: {stats['Best Trade [%]']}")
print(f"Worst Trade [%]: {stats['Worst Trade [%]']}")
print(f"Avg Winning Trade [%]: {stats['Avg Winning Trade [%]']}")
print(f"Avg Losing Trade [%]: {stats['Avg Losing Trade [%]']}")
print(f"Avg Winning Trade Duration: {stats['Avg Winning Trade Duration']}")
print(f"Avg Losing Trade Duration: {stats['Avg Losing Trade Duration']}")
print(f"Profit Factor: {stats['Profit Factor']}")
print(f"Expectancy: {stats['Expectancy']}")



✅ Modelo cargado: best_model_walkforward_weekly_dt_fullparams.joblib

📊 Estadísticas del portafolio (con modelo):
Start Value: 10000.0
End Value: 8935.843149550032
Total Return [%]: -10.641568504499679
Benchmark Return [%]: 5.855332293093851
Max Gross Exposure [%]: 44.956059874057786
Total Fees Paid: 708.83790045
Max Drawdown [%]: 10.831031158650282
Max Drawdown Duration: 6020.0
Total Trades: 92.0
Total Closed Trades: 92.0
Total Open Trades: 0.0
Open Trade PnL: 0.0
Win Rate [%]: 19.565217391304348
Best Trade [%]: 1.4147208978822685
Worst Trade [%]: -2.292652487885969
Avg Winning Trade [%]: 0.31612466387017
Avg Losing Trade [%]: -0.45147196020192776
Avg Winning Trade Duration: 33.666666666666664
Avg Losing Trade Duration: 23.743243243243242
Profit Factor: 0.17128085111923305
Expectancy: -11.566922287499624


c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [5]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"  # ✅ Desactivar Numba JIT antes de todo

import pandas as pd
import numpy as np

# === 1. Cargar df_test ===
df_test = pd.read_csv("df_test.csv").reset_index(drop=True)

# Agregar columnas temporales si existe 'time'
if 'time' in df_test.columns:
    df_test['time'] = pd.to_datetime(df_test['time'])
    df_test['hour'] = df_test['time'].dt.hour
    df_test['weekday'] = df_test['time'].dt.weekday

# === 2. Simulación sin modelo (solo Renko o MRB bar logic) ===
entries = (df_test['signal'] == 1).fillna(False).astype(bool)

# Parámetros óptimos encontrados anteriormente
tp_pips = 0.00050
horizon = 48

# === 3. Generar señales de salida (TP o timeout) ===
exits = pd.Series(False, index=df_test.index)

for i in entries[entries].index:
    entry_price = df_test.at[i, 'close']
    future = df_test.iloc[i+1:min(i+1+horizon, len(df_test))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        fallback_idx = future.index[-1]
        exits.at[fallback_idx] = True

# === 4. Preprocesamiento final ===
prices = df_test['close'].astype(float).ffill().bfill()
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

# === 5. Configuración de trading ===
init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = float(lot_size * leverage)

# === 6. Cálculo del rendimiento manual ===
capital = init_cash
positions = []
equity_curve = []
total_trades = 0
total_closed_trades = 0
win_trades = 0
loss_trades = 0
total_fees = 0

# Iteración sobre las señales para calcular el rendimiento
for i in range(len(prices)):
    if entries[i]:
        positions.append(prices[i])  # Entrada al mercado
    elif exits[i] and positions:
        entry_price = positions.pop()  # Salida del mercado
        trade_pnl = (prices[i] - entry_price) * fixed_size  # Ganancia o pérdida de la operación
        capital += trade_pnl
        
        # Calcular el fee
        fee = abs(trade_pnl) * 0.001
        total_fees += fee
        
        # Contabilizar el número de operaciones
        total_trades += 1
        total_closed_trades += 1
        
        # Contabilizar si fue una operación ganadora o perdedora
        if trade_pnl > 0:
            win_trades += 1
        else:
            loss_trades += 1

    equity_curve.append(capital)

# === 7. Cálculo de estadísticas de rendimiento ===

# Total return
total_return = (capital - init_cash) / init_cash * 100

# Max Drawdown
equity_curve = np.array(equity_curve)
peak = np.maximum.accumulate(equity_curve)
drawdown = (equity_curve - peak) / peak * 100
max_drawdown = drawdown.min()

# Win Rate
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0

# Promedio de operaciones ganadoras y perdedoras
avg_winning_trade = (capital - init_cash) / win_trades if win_trades > 0 else 0
avg_losing_trade = (init_cash - capital) / loss_trades if loss_trades > 0 else 0

# Duración promedio de operaciones ganadoras y perdedoras (en días)
# Usamos la duración en días (diferencia entre las fechas de entrada y salida)
winning_trade_durations = []
losing_trade_durations = []

# Asumimos que las señales de entrada y salida ocurren en el mismo día para simplificar
for i in range(len(prices)):
    if entries[i]:
        entry_date = df_test.at[i, 'time']
    elif exits[i] and entry_date:
        exit_date = df_test.at[i, 'time']
        duration = (exit_date - entry_date).days
        if (prices[i] - prices[entries.idxmax()]) > 0:
            winning_trade_durations.append(duration)
        else:
            losing_trade_durations.append(duration)

avg_winning_trade_duration = np.mean(winning_trade_durations) if winning_trade_durations else 0
avg_losing_trade_duration = np.mean(losing_trade_durations) if losing_trade_durations else 0

# === 8. Resultados ===
print("\n📊 Estadísticas del rendimiento del backtest usando Renko:")
print(f"Capital final: {capital:.2f}")
print(f"Ganancia/Pérdida neta: {capital - init_cash:.2f}")
print(f"Retorno total [%]: {total_return:.2f}")
print(f"Max Drawdown [%]: {max_drawdown:.2f}")
print(f"Total Fees Paid: {total_fees:.2f}")
print(f"Total Trades: {total_trades}")
print(f"Total Closed Trades: {total_closed_trades}")
print(f"Win Rate [%]: {win_rate:.2f}")
print(f"Best Trade [%]: {avg_winning_trade:.2f}")
print(f"Worst Trade [%]: {avg_losing_trade:.2f}")
print(f"Avg Winning Trade Duration: {avg_winning_trade_duration:.2f} days")
print(f"Avg Losing Trade Duration: {avg_losing_trade_duration:.2f} days")

# Estadísticas adicionales
profit_factor = sum([capital - init_cash for capital in equity_curve if capital > init_cash]) / abs(sum([capital - init_cash for capital in equity_curve if capital < init_cash]))
expectancy = (win_trades / total_trades) * avg_winning_trade - (loss_trades / total_trades) * avg_losing_trade

print(f"Profit Factor: {profit_factor:.2f}")
print(f"Expectancy: {expectancy:.2f}")




📊 Estadísticas del rendimiento del backtest usando Renko:
Capital final: 8781.85
Ganancia/Pérdida neta: -1218.15
Retorno total [%]: -12.18
Max Drawdown [%]: -12.30
Total Fees Paid: 1.60
Total Trades: 163
Total Closed Trades: 163
Win Rate [%]: 44.79
Best Trade [%]: -16.69
Worst Trade [%]: 13.53
Avg Winning Trade Duration: 0.72 days
Avg Losing Trade Duration: 0.81 days
Profit Factor: 0.00
Expectancy: -14.95


In [ ]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
import joblib
import vectorbt as vbt
import ta

from datetime import timedelta

# === 1. Cargar df_test original y calcular estadísticos base ===
df_base = pd.read_csv("df_test.csv")
df_base['time'] = pd.to_datetime(df_base['time'])

n_rows = len(df_base)
np.random.seed(42)

# === 2. Simulación Monte Carlo de precios OHLC ===
close_sim = np.cumsum(np.random.normal(0, 0.0008, n_rows)) + df_base['close'].iloc[0]
open_sim = close_sim + np.random.normal(0, 0.0003, n_rows)
high_sim = np.maximum(open_sim, close_sim) + np.abs(np.random.normal(0, 0.0005, n_rows))
low_sim = np.minimum(open_sim, close_sim) - np.abs(np.random.normal(0, 0.0005, n_rows))
volume_sim = np.abs(np.random.normal(1000, 300, n_rows))

df_sim = pd.DataFrame({
    'time': pd.date_range(start='2023-01-01', periods=n_rows, freq='H'),
    'open': open_sim,
    'high': high_sim,
    'low': low_sim,
    'close': close_sim,
    'tick_volume': volume_sim
})

df_sim['hour'] = df_sim['time'].dt.hour
df_sim['weekday'] = df_sim['time'].dt.weekday

# === 3. Calcular las 14 features técnicas básicas ===
df_sim['sma_200_vs_100'] = ta.trend.sma_indicator(df_sim['close'], 200) - ta.trend.sma_indicator(df_sim['close'], 100)
df_sim['sma_50_vs_25'] = ta.trend.sma_indicator(df_sim['close'], 50) - ta.trend.sma_indicator(df_sim['close'], 25)
df_sim['rsi_14'] = ta.momentum.rsi(df_sim['close'], window=14)
df_sim['acceleration_12'] = df_sim['close'].diff().diff(periods=12)
macd = ta.trend.macd_diff(df_sim['close'])
df_sim['macd'] = macd
df_sim['stoch_k'] = ta.momentum.stoch(df_sim['high'], df_sim['low'], df_sim['close'])
df_sim['momentum_12'] = df_sim['close'].diff(periods=12)
bb = ta.volatility.BollingerBands(df_sim['close'], window=20)
df_sim['bb_middle'] = bb.bollinger_mavg()
df_sim['bb_upper'] = bb.bollinger_hband()
df_sim['bb_lower'] = bb.bollinger_lband()
df_sim['weighted_close'] = (df_sim['high'] + df_sim['low'] + 2 * df_sim['close']) / 4
df_sim['poc_12'] = df_sim['close'].rolling(12).mean()  # Proxy for POC
df_sim['high_channel_20'] = df_sim['high'].rolling(20).max()
df_sim['low_channel_20'] = df_sim['low'].rolling(20).min()
df_sim['abs_diff_mrb_vs_close'] = np.abs(df_sim['close'] - df_sim['close'].rolling(20).mean())

# === 4. Simular variables Renko (últimos 5 precios en señales)
for i in range(1, 6):
    df_sim[f'last_signal_price_{i}'] = df_sim['close'].shift(i)

# === 5. Simular columna de señal activa ===
df_sim['signal'] = np.random.choice([0, 1], size=n_rows, p=[0.8, 0.2])  # solo algunas señales activas

# === 6. Preparar features y cargar modelo ===
features_cols = [
    'close', 'sma_200_vs_100', 'sma_50_vs_25', 'rsi_14', 'acceleration_12',
    'macd', 'stoch_k', 'momentum_12', 'bb_middle', 'bb_upper',
    'bb_lower', 'weighted_close', 'poc_12', 'high_channel_20',
    'low_channel_20', 'abs_diff_mrb_vs_close',
    'last_signal_price_1', 'last_signal_price_2',
    'last_signal_price_3', 'last_signal_price_4',
    'last_signal_price_5', 'hour', 'weekday'
]

modelo_path = "best_model_walkforward_weekly_dt_fullparams.joblib"
model = joblib.load(modelo_path)

# === 7. Predicción y probabilidad ===
X_sim = df_sim[features_cols].fillna(0)
df_sim['pred'] = model.predict(X_sim)
df_sim['proba_1'] = model.predict_proba(X_sim)[:, 1]

# === 8. Señales de entrada y salida (TP o N velas)
entries = ((df_sim['signal'] == 1) & (df_sim['pred'] == 1)).fillna(False)
tp_pips = 0.0012
horizon = 20
exits = pd.Series(False, index=df_sim.index)

for i in entries[entries].index:
    entry_price = df_sim.at[i, 'close']
    future = df_sim.iloc[i+1:min(i+1+horizon, len(df_sim))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        exits.at[future.index[-1]] = True

# === 9. Simulación en VectorBT ===
prices = df_sim['close'].astype(float)
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = lot_size * leverage

portfolio = vbt.Portfolio.from_signals(
    close=prices,
    entries=entries,
    exits=exits,
    size=fixed_size,
    init_cash=init_cash,
    fees=0.001,
    slippage=0.0005,
    direction='longonly'
)

# === 10. Resultados
print("\n📊 Simulación con Monte Carlo:")
stats = portfolio.stats()

# Mostrar estadísticas clave como en el primer código
print(f"Start Value: {stats['Start Value']}")
print(f"End Value: {stats['End Value']}")
print(f"Total Return [%]: {stats['Total Return [%]']}")
print(f"Benchmark Return [%]: {stats['Benchmark Return [%]']}")
print(f"Max Gross Exposure [%]: {stats['Max Gross Exposure [%]']}")
print(f"Total Fees Paid: {stats['Total Fees Paid']}")
print(f"Max Drawdown [%]: {stats['Max Drawdown [%]']}")
print(f"Max Drawdown Duration: {stats['Max Drawdown Duration']}")
print(f"Total Trades: {stats['Total Trades']}")
print(f"Total Closed Trades: {stats['Total Closed Trades']}")
print(f"Total Open Trades: {stats['Total Open Trades']}")
print(f"Open Trade PnL: {stats['Open Trade PnL']}")
print(f"Win Rate [%]: {stats['Win Rate [%]']}")
print(f"Best Trade [%]: {stats['Best Trade [%]']}")
print(f"Worst Trade [%]: {stats['Worst Trade [%]']}")
print(f"Avg Winning Trade [%]: {stats['Avg Winning Trade [%]']}")
print(f"Avg Losing Trade [%]: {stats['Avg Losing Trade [%]']}")
print(f"Avg Winning Trade Duration: {stats['Avg Winning Trade Duration']}")
print(f"Avg Losing Trade Duration: {stats['Avg Losing Trade Duration']}")
print(f"Profit Factor: {stats['Profit Factor']}")
print(f"Expectancy: {stats['Expectancy']}")

# === 11. Gráficos
portfolio.plot(title="Simulación - Monte Carlo").show()



C:\Users\sebas\AppData\Local\Temp\ipykernel_23028\2834089440.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'time': pd.date_range(start='2023-01-01', periods=n_rows, freq='H'),



📊 Simulación con Monte Carlo:


c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start Value: 10000.0
End Value: 3957.343585669034
Total Return [%]: -60.426564143309655
Benchmark Return [%]: -0.5444197433948306
Max Gross Exposure [%]: 94.52143472747349
Total Fees Paid: 4223.7871489945555
Max Drawdown [%]: 60.42656414330973
Max Drawdown Duration: 6168.0
Total Trades: 544.0
Total Closed Trades: 544.0
Total Open Trades: 0.0
Open Trade PnL: 0.0
Win Rate [%]: 0.1838235294117647
Best Trade [%]: 0.05189697063618019
Worst Trade [%]: -0.9607398226236387
Avg Winning Trade [%]: 0.05189697063618019
Avg Losing Trade [%]: -0.28667843189398984
Avg Winning Trade Duration: 12.0
Avg Losing Trade Duration: 5.895027624309392
Profit Factor: 0.0003284901337669377
Expectancy: -11.107824291049548
